In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['ali.txt']


In [2]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
import string
import re

Using TensorFlow backend.


In [3]:
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [4]:
def load_doc(filename):
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
    

In [5]:
in_filename='../input/ali.txt'

In [6]:
doc=load_doc(in_filename)

In [7]:
print(doc[0:100])

In a town of Persia lived two brothers, sons of a poor man; one named
Cassim, the other Ali Baba. Ca


In [8]:
def clean_doc(doc):
    doc = doc.replace('--', ' ')
    tokens = doc.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [9]:
tokens=clean_doc(doc)
tokens[0:50]


['in',
 'a',
 'town',
 'of',
 'persia',
 'lived',
 'two',
 'brothers',
 'sons',
 'of',
 'a',
 'poor',
 'man',
 'one',
 'named',
 'cassim',
 'the',
 'other',
 'ali',
 'baba',
 'cassim',
 'the',
 'elder',
 'married',
 'a',
 'wife',
 'with',
 'a',
 'considerable',
 'fortune',
 'and',
 'lived',
 'at',
 'his',
 'ease',
 'but',
 'the',
 'wife',
 'of',
 'ali',
 'baba',
 'was',
 'as',
 'poor',
 'as',
 'himself',
 'they',
 'dwelt',
 'in',
 'a']

In [10]:
tk=len(tokens)
tk

957

In [11]:
length= 50 + 1
sequence=[]
for i in range(length,tk):
    seq=tokens[i-length:i]
    line=' '.join(seq)
    sequence.append(line)
    

In [12]:
sequence

['in a town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a considerable fortune and lived at his ease but the wife of ali baba was as poor as himself they dwelt in a mean',
 'a town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a considerable fortune and lived at his ease but the wife of ali baba was as poor as himself they dwelt in a mean cottage',
 'town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a considerable fortune and lived at his ease but the wife of ali baba was as poor as himself they dwelt in a mean cottage in',
 'of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a considerable fortune and lived at his ease but the wife of ali baba was as poor as himself they dwelt in a mean cotta

In [13]:
out_filename = 'ali1.txt'
save_doc(sequence, out_filename)

In [14]:
len(sequence)

906

In [15]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(sequence)

In [16]:
sequence1 = tokenizer.texts_to_sequences(sequence)

In [17]:
sequence1

[[10,
  6,
  362,
  4,
  361,
  346,
  78,
  136,
  360,
  4,
  6,
  80,
  74,
  26,
  358,
  44,
  1,
  134,
  8,
  9,
  44,
  1,
  356,
  354,
  6,
  75,
  15,
  6,
  352,
  351,
  2,
  346,
  24,
  7,
  348,
  347,
  1,
  75,
  4,
  8,
  9,
  12,
  13,
  80,
  13,
  43,
  130,
  129,
  10,
  6,
  138],
 [6,
  362,
  4,
  361,
  346,
  78,
  136,
  360,
  4,
  6,
  80,
  74,
  26,
  358,
  44,
  1,
  134,
  8,
  9,
  44,
  1,
  356,
  354,
  6,
  75,
  15,
  6,
  352,
  351,
  2,
  346,
  24,
  7,
  348,
  347,
  1,
  75,
  4,
  8,
  9,
  12,
  13,
  80,
  13,
  43,
  130,
  129,
  10,
  6,
  138,
  139],
 [362,
  4,
  361,
  346,
  78,
  136,
  360,
  4,
  6,
  80,
  74,
  26,
  358,
  44,
  1,
  134,
  8,
  9,
  44,
  1,
  356,
  354,
  6,
  75,
  15,
  6,
  352,
  351,
  2,
  346,
  24,
  7,
  348,
  347,
  1,
  75,
  4,
  8,
  9,
  12,
  13,
  80,
  13,
  43,
  130,
  129,
  10,
  6,
  138,
  139,
  10],
 [4,
  361,
  346,
  78,
  136,
  360,
  4,
  6,
  80,
  74,
  26,
  358,
  

In [18]:
vocab_size=len(tokenizer.word_index) + 1
print(vocab_size)


364


In [19]:
sequence1=np.array(sequence1)

In [20]:
sequence1

array([[ 10,   6, 362, ...,  10,   6, 138],
       [  6, 362,   4, ...,   6, 138, 139],
       [362,   4, 361, ..., 138, 139,  10],
       ...,
       [ 30, 338, 339, ...,  10, 137,   4],
       [338, 339, 340, ..., 137,   4,   7],
       [339, 340, 341, ...,   4,   7, 363]])

In [21]:
x,y=sequence1[:,:-1],sequence1[:,-1]

In [22]:
y = to_categorical(y, num_classes=vocab_size)

In [23]:
print("shape of training datasets(x,y)=",x.shape,y.shape)

shape of training datasets(x,y)= (906, 50) (906, 364)


In [24]:
def define_model(vocab_size, seq_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 50, input_length=seq_length))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [25]:
x.shape[1]

50

In [26]:
model=define_model(vocab_size,x.shape[1])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            18200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 364)               36764     
Total params: 205,864
Trainable params: 205,864
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(x,y,batch_size=128,epochs=170)

Epoch 1/170
906/906 [==============================] - 2s 2ms/step - loss: 5.8930 - accuracy: 0.0464
Epoch 2/170
906/906 [==============================] - 1s 947us/step - loss: 5.7847 - accuracy: 0.0828
Epoch 3/170
906/906 [==============================] - 1s 856us/step - loss: 5.4536 - accuracy: 0.0828
Epoch 4/170
906/906 [==============================] - 1s 895us/step - loss: 5.2845 - accuracy: 0.0828
Epoch 5/170
906/906 [==============================] - 1s 840us/step - loss: 5.2196 - accuracy: 0.0828
Epoch 6/170
906/906 [==============================] - 1s 917us/step - loss: 5.1824 - accuracy: 0.0828
Epoch 7/170
906/906 [==============================] - 1s 862us/step - loss: 5.1662 - accuracy: 0.0828
Epoch 8/170
906/906 [==============================] - 1s 852us/step - loss: 5.1594 - accuracy: 0.0828
Epoch 9/170
906/906 [==============================] - 1s 821us/step - loss: 5.1572 - accuracy: 0.0828
Epoch 10/170
906/906 [==============================] - 1s 828us/step - los

In [28]:
sequence

['in a town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a considerable fortune and lived at his ease but the wife of ali baba was as poor as himself they dwelt in a mean',
 'a town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a considerable fortune and lived at his ease but the wife of ali baba was as poor as himself they dwelt in a mean cottage',
 'town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a considerable fortune and lived at his ease but the wife of ali baba was as poor as himself they dwelt in a mean cottage in',
 'of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a considerable fortune and lived at his ease but the wife of ali baba was as poor as himself they dwelt in a mean cotta

In [29]:
from keras.models import load_model

In [30]:
model.save('model.h5')

In [31]:
model = load_model('model.h5')

In [32]:
sequence

['in a town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a considerable fortune and lived at his ease but the wife of ali baba was as poor as himself they dwelt in a mean',
 'a town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a considerable fortune and lived at his ease but the wife of ali baba was as poor as himself they dwelt in a mean cottage',
 'town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a considerable fortune and lived at his ease but the wife of ali baba was as poor as himself they dwelt in a mean cottage in',
 'of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a considerable fortune and lived at his ease but the wife of ali baba was as poor as himself they dwelt in a mean cotta

In [33]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
            # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)
# load cleaned text sequences
in_filename = 'ali1.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
# select a seed text
seed_text = 'Ali Baba told his brother the secret of the cave. Cassim rose early next morning, and set out with ten mules loaded with great chests.'
print(seed_text + '\n')
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)


Ali Baba told his brother the secret of the cave. Cassim rose early next morning, and set out with ten mules loaded with great chests.

and the robbers coming said full the secret of the garden of the robbers came to as out a man shut to adopt baba dwelt sir said to table great way at illustration illustration ali baba was led blindfolded the same way said having said two pieces and hid himself
